# Install Package and Manage Data

In [ ]:
# !pip install Sastrawi
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 
import re

import random
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
 
 
import nltk
from nltk.corpus import stopwords
 
from tqdm import tqdm
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
# df = pd.read_csv("/content/drive/MyDrive/train.csv", usecols=["label", "reviewText"])
# with tf.device('/device:GPU:0'):
#df = pd.read_csv("/content/drive/MyDrive/train.csv", usecols=["label", "reviewText"])
#import tensorflow as tf
df = pd.read_csv("/content/drive/MyDrive/Big Data Eka/Train1_stemmed_lemmatized_no_extra_stops.csv", usecols=["label", "text_lemmatize" ])

In [ ]:
df

,label,text_lemmatize
0,2,sound track beautiful paint senery mind well w...
1,2,im reading lot review saying best game soundtr...
2,2,soundtrack favorite music time hand intense sa...
3,2,truly like soundtrack enjoy video game music p...
4,2,youve played game know divine music every sing...
...,...,...
399993,1,sure trainer super easy set holy god loud know...
399994,2,sock aid cloth covered great hose sock tended ...
399995,1,book frustrating put japanese text tiny letter...
399996,2,came across book vacation cabin minnesota read...


In [ ]:
df.isnull().sum()

label             0
text_lemmatize    0
dtype: int64

# PYSPARK

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import Row
# from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.feature import CountVectorizer, StopWordsRemover, IDF
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes

# Menyiapkan Data

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
# sc = SparkContext.getOrCreate()
sc = SparkSession.builder.appName('SentimentClassifier').getOrCreate()
spark = SparkSession(sc)
# documents = sc.dataframe("/content/drive/MyDrive/train.csv").map(lambda line: line.split(" "))
# save data https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html
dfSPARK = spark.read.load("/content/drive/MyDrive/Big Data Eka/Train1_stemmed_lemmatized_no_extra_stops.csv",
                     format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
datafix.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|          reviewText|       selected_text|         text_porter|       text_snowball|      text_lemmatize|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|    2|This sound track ...|sound track beaut...|sound track beaut...|sound track beaut...|sound track beaut...|
|    2|I'm reading a lot...|im reading lot re...|im read lot revie...|im read lot revie...|im reading lot re...|
|    2|"This soundtrack ...| ""Time of the Dr...| and ""Chronomant...| probably the bes...| so I can't say f...|
|    2|I truly like this...|truly like soundt...|truli like soundt...|truli like soundt...|truly like soundt...|
|    2|If you've played ...|youve played game...|youv play game kn...|youv play game kn...|youve played game...|
|    2|I am quite sure a...|quite sure actual...|quit sure actual ...|quit sure actual ...|quite

# Tokenize Menggunakan Pyspark

In [ ]:
# tokenizer = Tokenizer(inputCol="reviewText", outputCol="tokenText")
# categoryIndexer = StringIndexer(inputCol="category", outputCol="label")

stages = []
# 1. clean data and tokenize sentences using RegexTokenizer
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="tokens")
stages += [regexTokenizer]

# 2. CountVectorize the data
cv = CountVectorizer(inputCol="tokens", outputCol="token_features", minDF=2.0)#, vocabSize=3, minDF=2.0
stages += [cv]

# 3. Vectorise features using vectorassembler
vecAssembler = VectorAssembler(inputCols=['token_features'], outputCol="features")
stages += [vecAssembler]

# 4. Convert the labels to numerical values using binariser
indexer = StringIndexer(inputCol="label", outputCol="labelpredict")
stages += [indexer]

[print('\n', stage) for stage in stages]
# wordsData = tokenizer.transform(df)
# regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="tokens", pattern="\\W+")
# wordsData = regexTokenizer.transform(df)
#https://stackoverflow.com/questions/41352267/creating-tf-idf-vector-from-a-spark-dataframe-with-text-column


 RegexTokenizer_8a02666b5b90

 CountVectorizer_6f34d480554d

 VectorAssembler_f8330d90d6ef

 StringIndexer_476e8cc14bf2


[None, None, None, None]

In [ ]:
datafix.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|          reviewText|       selected_text|         text_porter|       text_snowball|      text_lemmatize|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|    2|This sound track ...|sound track beaut...|sound track beaut...|sound track beaut...|sound track beaut...|
|    2|I'm reading a lot...|im reading lot re...|im read lot revie...|im read lot revie...|im reading lot re...|
|    2|"This soundtrack ...| ""Time of the Dr...| and ""Chronomant...| probably the bes...| so I can't say f...|
|    2|I truly like this...|truly like soundt...|truli like soundt...|truli like soundt...|truly like soundt...|
|    2|If you've played ...|youve played game...|youv play game kn...|youv play game kn...|youve played game...|
|    2|I am quite sure a...|quite sure actual...|quit sure actual ...|quit sure actual ...|quite

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)
data = pipeline.fit(datafix).transform(datafix)

In [ ]:
data = datafix.drop("reviewText").drop("label").drop("tokens").drop("token_features")

+--------------------+-----+--------------------+--------------------+--------------------+------------+
|          reviewText|label|              tokens|      token_features|            features|labelpredict|
+--------------------+-----+--------------------+--------------------+--------------------+------------+
|This sound track ...|    2|[this, sound, tra...|(226435,[0,1,2,3,...|(226435,[0,1,2,3,...|         0.0|
|I'm reading a lot...|    2|[i'm, reading, a,...|(226435,[0,1,2,3,...|(226435,[0,1,2,3,...|         0.0|
|"This soundtrack ...|    2|["this, soundtrac...|(226435,[0,1,4,5,...|(226435,[0,1,4,5,...|         0.0|
|I truly like this...|    2|[i, truly, like, ...|(226435,[0,1,2,3,...|(226435,[0,1,2,3,...|         0.0|
|If you've played ...|    2|[if, you've, play...|(226435,[0,1,3,5,...|(226435,[0,1,3,5,...|         0.0|
|I am quite sure a...|    2|[i, am, quite, su...|(226435,[0,1,2,3,...|(226435,[0,1,2,3,...|         0.0|
|"This is a self-p...|    1|["this, is, a, se...|(22643

In [ ]:
newDF = data.drop("reviewText").drop("label").drop("tokens").drop("token_features")
dataSpark = newDF.select("features","labelpredict")

In [ ]:
print(newDF.show())

+------------+--------------------+
|labelpredict|            features|
+------------+--------------------+
|         0.0|(226435,[0,1,2,3,...|
|         0.0|(226435,[0,1,2,3,...|
|         0.0|(226435,[0,1,4,5,...|
|         0.0|(226435,[0,1,2,3,...|
|         0.0|(226435,[0,1,3,5,...|
|         0.0|(226435,[0,1,2,3,...|
|         1.0|(226435,[0,1,2,3,...|
|         0.0|(226435,[0,1,2,3,...|
|         0.0|(226435,[0,1,2,3,...|
|         0.0|(226435,[0,1,2,3,...|
|         1.0|(226435,[1,2,3,4,...|
|         0.0|(226435,[0,1,3,4,...|
|         0.0|(226435,[0,1,2,4,...|
|         1.0|(226435,[0,1,2,3,...|
|         1.0|(226435,[0,1,2,3,...|
|         1.0|(226435,[0,1,5,7,...|
|         0.0|(226435,[0,3,6,7,...|
|         0.0|(226435,[0,1,2,3,...|
|         0.0|(226435,[0,1,2,3,...|
|         1.0|(226435,[0,1,2,4,...|
+------------+--------------------+
only showing top 20 rows

None


# Bagi Data Train dan Test

In [ ]:
# train, test = newDF.randomSplit([0.8, 0.], seed = 2018)
train, test = newDF.randomSplit([0.8, 0.])

NameError: ignored

# Naive Bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes
# Initialise the model
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
# Fit the model
model = nb.fit(train)
# Make predictions on test data
NBpredictions = model.transform(test)
NBpredictions.select("features", "labelnew", "probability").show()

+--------+----------+--------------------+
|labelnew|prediction|         probability|
+--------+----------+--------------------+
|     0.0|       0.0|[0.99992733889387...|
|     0.0|       1.0|[0.00715290898721...|
|     0.0|       1.0|[1.88133976674918...|
|     1.0|       1.0|[0.45302600075835...|
|     0.0|       1.0|[5.20235844022676...|
|     0.0|       1.0|[0.01316379916471...|
|     1.0|       1.0|[0.09898257872793...|
|     1.0|       1.0|[0.01664370077268...|
|     0.0|       1.0|[0.39148136961819...|
|     1.0|       1.0|[0.45571448426457...|
|     1.0|       0.0|[0.65698762839194...|
|     1.0|       1.0|[0.38443304774751...|
|     0.0|       1.0|[4.63150617133155...|
|     0.0|       1.0|[0.22612424007475...|
|     0.0|       1.0|[0.13615854042327...|
|     1.0|       0.0|[0.88102546893807...|
|     0.0|       1.0|[0.01473924880637...|
|     0.0|       1.0|[0.00553262840617...|
|     0.0|       1.0|[0.03133216010516...|
|     0.0|       1.0|[0.01247002391003...|
+--------+-

# Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Initialise the model
Lr = LogisticRegression(labelCol="labelnew", featuresCol="features", maxIter=10,regParam=0.3)
# Fit the model
model = Lr.fit(train)
# Make predictions on test data
LRpredictions = model.transform(test)
LRpredictions.select("features", "labelnew", "probability").show()

# Random Forest Classifier

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# Initialise the model
Rfc = RandomForestClassifier(numTrees=3, maxDepth=5,seed=42,labelCol="label",featuresCol="features")
# Fit the model
model = Rfc.fit(train)
# Make predictions on test data
RFCpredictions = model.transform(test)
RFCpredictions.select("features", "labelnew", "probability").show()

# Evaluasi Akurasi Model

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print ("Model Accuracy: ", accuracy)

Model Accuracy:  1.0
